## 1. 환경설정

In [21]:
import json

from dotenv import load_dotenv

load_dotenv('../.env')

True

## 2. 다양한 문서 형식 처리하기
### 2.1 PDF 문설

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("./data/transformer.pdf")
pdf_docs = pdf_loader.load()
print(len(pdf_docs))
print(pdf_docs)
print(pdf_docs[0].metadata)
print(pdf_docs[0].page_content)


## 2.2 웹 문서

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])
web_docs = web_loader.load()
print(len(web_docs))
web_docs[0].metadata
print(web_docs[0].page_content)

## 2.3 JSON 파일

In [42]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[].content",  # messages 배열의 content 필드만 추출
    text_content=True  # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수: ", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수:  5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [43]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",    # messages 배열의 모든 아이템을 추출
    text_content=False,          # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 {"sender": "\uae40\ucca0\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\uc548\ub155\ud558\uc138\uc694 \uc5ec\ub7ec\ubd84, \uc624\ub298 \ud68c\uc758 \uc2dc\uac04 \ud655\uc778\ucc28 \uc5f0\ub77d\ub4dc\ub9bd\ub2c8\ub2e4."}


In [44]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩되어 있음)
from langchain_core.documents import Document

decoded_json_docs = []
for doc in json_docs:

    decoded_data = json.loads(doc.page_content)

    # decoded_json_docs.append({
    #     "metadata": doc.metadata,
    #     "page_content": json.dumps(decoded_data, ensure_ascii=False)
    # })

    document_obj = Document(page_content=json.dumps(decoded_data, ensure_ascii=False), metadata=doc.metadata)
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
# print("처음 문서의 메타데이터: \n", decoded_json_docs[0]["metadata"])
print("처음 문서의 메타데이터: \n", decoded_json_docs[0].metadata)
print("-" * 50)
# print("처음 문서의 내용: \n", decoded_json_docs[0]["page_content"])
print("처음 문서의 내용: \n", decoded_json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 {"sender": "김철수", "timestamp": "2023-09-15 09:30:22", "content": "안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다."}


In [45]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata


json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",
    content_key="content",
    metadata_func=metadata_func,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [46]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.jsonl', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [ ]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata

json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    metadata_func=metadata_func,
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)